In [1]:
import pandas as pd
import numpy as np
import requests
import json
from rapidfuzz import fuzz

In [2]:
lnos = pd.read_csv('lnos.csv')
lnos

,country,genre,song name,artist,rating
0,antarctica,indie rock,how many people,nunatuk,6.5
1,argentina,indie rock,bichos,homogenica,7.0
2,argentina,indie rock,turista,homogenica,8.0
3,argentina,indie rock,animales,homogenica,8.0
4,argentina,indie rock,ratas,homogenica,8.5
...,...,...,...,...,...
185,uzbekistan,indie rock,Нам не нужны друзья,wladek sheen,7.0
186,venezuela,sifri rock,retroceder,los mesoneros,6.5
187,vietnam,indie rock,hay la,"tora drums, hoanghieu, drumc",8.5
188,vietnam,indie rock,nho em,do it dad,9.0


## Make API Request

In [3]:
import re
import time

In [4]:
def remove_punctuation(input_string):
    no_punc= re.sub(r'[^\w\s]', '', input_string)
    return re.sub(r'\s+', ' ', no_punc).strip()


In [5]:
# delete apostrophes and commas
remove_punctuation('empina o fuzil // power trip')

'empina o fuzil power trip'

In [6]:
import time
import requests
from rapidfuzz import fuzz

minidata = lnos[:]
album_urls = []
preview_urls = []
responses = []

for i in range(len(minidata)):
    if (i + 1) % 49 == 0:
        time.sleep(5)

    # Create search term
    songName = '+'.join(remove_punctuation(minidata.iloc[i]['song name']).split())
    artistName = '+'.join(remove_punctuation(minidata.iloc[i]['artist']).split())
    term = songName + '+' + artistName
    print(term)

    # Make request to Deezer API (increase limit to allow better matching)
    url = f'https://api.deezer.com/search?q={term}&limit=5'
    response = requests.get(url)
    print(response)

    album_url = None
    preview_url = None

    if response.status_code == 200:
        response_json = response.json()
        responses.append(response_json)
        candidates = response_json.get('data', [])
        
        best_score = 0
        best_item = None
        
        target_song = minidata.iloc[i]['song name'].lower().strip()
        target_artist = minidata.iloc[i]['artist'].lower().strip()

        for item in candidates:
            song_match = item['title'].lower().strip()
            artist_match = item['artist']['name'].lower().strip()
            song_score = fuzz.partial_ratio(target_song, song_match)
            artist_score = fuzz.partial_ratio(target_artist, artist_match)
            total_score = (song_score + artist_score) / 2

            if total_score > best_score:
                best_score = total_score
                best_item = item

        if best_score > 80 and best_item is not None:
            album_url = best_item['album']['cover_medium']
            preview_url = best_item['preview']

    else:
        responses.append('No Results')

    album_urls.append(album_url)
    preview_urls.append(preview_url)

how+many+people+nunatuk
<Response [200]>
bichos+homogenica
<Response [200]>
turista+homogenica
<Response [200]>
animales+homogenica
<Response [200]>
ratas+homogenica
<Response [200]>
somniferos+no+te+me+enojes
<Response [200]>
paraisos+mentales+marchitorial
<Response [200]>
the+noise+is+a+scream+clocker
<Response [200]>
stomp+the+revoir
<Response [200]>
acting+moderate+killed+by+9v+batteries
<Response [200]>
sheishob+din+raatri+karnival
<Response [200]>
Дажджамі+Бяз+Назвы
<Response [200]>
glow+slow+crush
<Response [200]>
histeria+llegas
<Response [200]>
shooters+internet+key
<Response [200]>
bathroom+wall+internet+key
<Response [200]>
i+could+cut+out+my+veins+internet+key
<Response [200]>
empina+o+fuzil+power+trip+isotopxs
<Response [200]>
fotos+de+uma+arvore+que+nao+parace+bem+micondria
<Response [200]>
i+luv+u+franklyn+glover
<Response [200]>
playboy+pusher174
<Response [200]>
de+areia+e+sal+sonhos+tomam+conta
<Response [200]>
hear+a+whisper+terraplana+winter
<Response [200]>
germabo

In [18]:
# response = requests.get('https://itunes.apple.com/search?term=montreal+40+malajube&limit=1')
# print(response.json())
# print(response)

In [7]:
responses

[{'data': [], 'total': 0},
 {'data': [{'id': 2670586662,
    'readable': True,
    'title': 'Bichos',
    'title_short': 'Bichos',
    'title_version': '',
    'link': 'https://www.deezer.com/track/2670586662',
    'duration': 199,
    'rank': 11656,
    'explicit_lyrics': False,
    'explicit_content_lyrics': 0,
    'explicit_content_cover': 2,
    'preview': 'https://cdnt-preview.dzcdn.net/api/1/1/b/d/a/0/bdacc8f47df13dbfc085a666a5abe10d.mp3?hdnea=exp=1745960877~acl=/api/1/1/b/d/a/0/bdacc8f47df13dbfc085a666a5abe10d.mp3*~data=user_id=0,application_id=42~hmac=1a0917623db49165fd1b779cb09abcd04af37ee251897cfcc4dea0de490698a7',
    'md5_image': '8a2531ad9ef62dff7d94a5970e235498',
    'artist': {'id': 248110192,
     'name': 'Homogénica',
     'link': 'https://www.deezer.com/artist/248110192',
     'picture': 'https://api.deezer.com/artist/248110192/image',
     'picture_small': 'https://cdn-images.dzcdn.net/images/artist/3bcea1fe7b55a09387407d8e896bde4c/56x56-000000-80-0-0.jpg',
     'pic

In [8]:
len(album_urls)

190

In [9]:
album_urls

[None,
 'https://cdn-images.dzcdn.net/images/cover/8a2531ad9ef62dff7d94a5970e235498/250x250-000000-80-0-0.jpg',
 'https://cdn-images.dzcdn.net/images/cover/3bcea1fe7b55a09387407d8e896bde4c/250x250-000000-80-0-0.jpg',
 'https://cdn-images.dzcdn.net/images/cover/8a2531ad9ef62dff7d94a5970e235498/250x250-000000-80-0-0.jpg',
 'https://cdn-images.dzcdn.net/images/cover/8a2531ad9ef62dff7d94a5970e235498/250x250-000000-80-0-0.jpg',
 'https://cdn-images.dzcdn.net/images/cover/2c598760d3613856b12043c980ba8609/250x250-000000-80-0-0.jpg',
 'https://cdn-images.dzcdn.net/images/cover/9ca7145143647548903c21fa29f113a2/250x250-000000-80-0-0.jpg',
 'https://cdn-images.dzcdn.net/images/cover/94302d493af6bef3cc053b221711ad95/250x250-000000-80-0-0.jpg',
 'https://cdn-images.dzcdn.net/images/cover/146c97d3990def991dddc21cc1ec5220/250x250-000000-80-0-0.jpg',
 'https://cdn-images.dzcdn.net/images/cover/2a537a3534339370da26a66122701a24/250x250-000000-80-0-0.jpg',
 'https://cdn-images.dzcdn.net/images/cover/8788

In [10]:
preview_urls

[None,
 'https://cdnt-preview.dzcdn.net/api/1/1/b/d/a/0/bdacc8f47df13dbfc085a666a5abe10d.mp3?hdnea=exp=1745960877~acl=/api/1/1/b/d/a/0/bdacc8f47df13dbfc085a666a5abe10d.mp3*~data=user_id=0,application_id=42~hmac=1a0917623db49165fd1b779cb09abcd04af37ee251897cfcc4dea0de490698a7',
 'https://cdnt-preview.dzcdn.net/api/1/1/7/9/5/0/7955249d698eb90fcbdcfd69b4c2e176.mp3?hdnea=exp=1745960877~acl=/api/1/1/7/9/5/0/7955249d698eb90fcbdcfd69b4c2e176.mp3*~data=user_id=0,application_id=42~hmac=8b6f23575c3e1daac08b6ab754cf3e692287f7e29ef49148939750c8d5b79626',
 'https://cdnt-preview.dzcdn.net/api/1/1/8/6/c/0/86c77f19915e8a5ac1decac4ab82c179.mp3?hdnea=exp=1745960877~acl=/api/1/1/8/6/c/0/86c77f19915e8a5ac1decac4ab82c179.mp3*~data=user_id=0,application_id=42~hmac=bdf36c73b7821dc22404c004fedf186425cb5d1acdc1cc396a5633c14a29bdcd',
 'https://cdnt-preview.dzcdn.net/api/1/1/e/6/5/0/e65f65d6f30e07f8d49e8960934fe797.mp3?hdnea=exp=1745960877~acl=/api/1/1/e/6/5/0/e65f65d6f30e07f8d49e8960934fe797.mp3*~data=user_id=0

## Add to lnos.csv

In [33]:
new_lnos = lnos.copy()
new_lnos['album_url']=pd.Series(album_urls)
new_lnos['preview_url']=pd.Series(preview_urls)

In [34]:
new_lnos.to_csv('lnos2.csv')